*Source of the case study:* https://8weeksqlchallenge.com/case-study-7/

---

## Case Study #7 - Balanced Tree Clothing Co. - Monthly report script

Below you can find a single SQL script that combines all of the questions into a scheduled report that the Balanced Tree team can run at the beginning of each month to calculate the previous month’s values. 

**!!!** To generate report for a particular month, they just **need to change the `month` and `year` values in a WHERE clause**.
<br>Currently, it will generate report for January of 2021.

--Let's create a table with product level information for all the transactions made for Balanced Tree in a particular month (e.g. January of 2021)

    CREATE TEMP TABLE sales_monthly AS
    (SELECT *
    FROM balanced_tree.sales
    WHERE EXTRACT(MONTH FROM start_txn_time) = 1 AND EXTRACT(YEAR FROM start_txn_time) = 2021);


--1. High Level Sales Analysis

--1.1. What was the total quantity sold for all products?

    SELECT SUM(qty) AS sold_products_qty
    FROM sales_monthly;


--1.2. What is the total generated revenue for all products before discounts?

    SELECT SUM(qty * price) AS total_revenue_before_discounts
    FROM sales_monthly;    


--1.3. What was the total discount amount for all products?

    SELECT ROUND(SUM(qty * price * discount::numeric/100), 2) AS total_discount
    FROM sales_monthly;


--2. Transaction Analysis

--2.1. How many unique transactions were there?

    SELECT COUNT(DISTINCT txn_id) AS transactions_cnt
    FROM sales_monthly;


--2.2. What is the average unique products purchased in each transaction?

    SELECT ROUND(AVG(products_txn_qty)) AS avg_products_qty
    FROM (SELECT SUM(qty) OVER (PARTITION BY txn_id) AS products_txn_qty
    	  FROM sales_monthly) t;


--2.3. What are the 25th, 50th and 75th percentile values for the revenue per transaction?

    WITH t AS
    (SELECT txn_id,
     		SUM(qty * price) AS txn_revenue
     FROM sales_monthly
     GROUP BY 1)
    SELECT percentile_cont(0.25) WITHIN GROUP (ORDER BY txn_revenue) AS revenue_25percentile,
           percentile_cont(0.5) WITHIN GROUP (ORDER BY txn_revenue) AS revenue_50percentile,
           percentile_cont(0.75) WITHIN GROUP (ORDER BY txn_revenue) AS revenue_75percentile
    FROM t;


--2.4. What is the average discount value per transaction?

    WITH t AS
    (SELECT txn_id,
    	    SUM(qty * price * discount::numeric/100) AS txn_discount
     FROM sales_monthly
     GROUP BY 1)
    SELECT ROUND(AVG(txn_discount), 2) AS avg_txn_discount
    FROM t;    


--2.5. What is the percentage split of all transactions for members vs non-members?

    SELECT member,
    	   ROUND(100*COUNT(member)::numeric / (SELECT COUNT(member) FROM sales_monthly), 2) AS member_percentage  
    FROM sales_monthly
    GROUP BY 1;


--2.6. What is the average revenue for member transactions and non-member transactions?

    SELECT member,
    	   SUM(qty * price) AS total_revenue_before_discounts 
    FROM sales_monthly
    GROUP BY 1; 


--3. Product Analysis

--3.1. What are the top 3 products by total revenue before discount?

    SELECT prod_id,
    	   product_name,
    	   SUM(qty * s.price) AS total_revenue_before_discounts 
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 3;


--3.2. What is the total quantity, revenue and discount for each segment?

    SELECT segment_id,
    	   segment_name,
           SUM(qty) AS total_qty,
    	   SUM(qty * s.price) AS total_revenue_before_discounts,
           ROUND(SUM(qty * s.price * discount::numeric/100), 2) AS total_discount
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2
    ORDER BY 1;


--3.3. What is the top selling product for each segment?

    WITH t AS
    (SELECT segment_id,
    	    segment_name,
            product_id,
            product_name,
            SUM(qty) AS total_qty,
            RANK() OVER (PARTITION BY segment_id ORDER BY SUM(qty) DESC) AS qty_rank
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2, 3, 4)
    SELECT segment_id,
    	   segment_name,
           product_id,
           product_name,
           total_qty
    FROM t
    WHERE qty_rank=1;


--3.4. What is the total quantity, revenue and discount for each category?

    SELECT category_id,
    	   category_name,
           SUM(qty) AS total_qty,
    	   SUM(qty * s.price) AS total_revenue_before_discounts,
           ROUND(SUM(qty * s.price * discount::numeric/100), 2) AS total_discount
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2
    ORDER BY 1;    


--3.5. What is the top selling product for each category?

    WITH t AS
    (SELECT category_id,
    	    category_name,
            product_id,
            product_name,
            SUM(qty) AS total_qty,
            RANK() OVER (PARTITION BY category_id ORDER BY SUM(qty) DESC) AS qty_rank
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2, 3, 4)
    SELECT category_id,
    	   category_name,
           product_id,
           product_name,
           total_qty
    FROM t
    WHERE qty_rank=1;  


--3.6. What is the percentage split of revenue by product for each segment?

    WITH t AS
    (SELECT segment_id,
    	    segment_name,
            product_id,
            product_name,
    	    SUM(qty * s.price) AS total_revenue_before_discounts
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2, 3, 4)
    SELECT *,
    	   ROUND(100*total_revenue_before_discounts / (SUM(total_revenue_before_discounts) OVER (PARTITION BY segment_id)), 2) AS revenue_percentage
    FROM t
    ORDER BY segment_id, revenue_percentage DESC;    


--3.7. What is the percentage split of revenue by segment for each category?

    WITH t AS
    (SELECT category_id,
    	    category_name,
            segment_id,
    	    segment_name,
    	    SUM(qty * s.price) AS total_revenue_before_discounts
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2, 3, 4)
    SELECT *,
    	   ROUND(100*total_revenue_before_discounts / (SUM(total_revenue_before_discounts) OVER (PARTITION BY category_id)), 2) AS revenue_percentage
    FROM t
    ORDER BY category_id, revenue_percentage DESC;   


--3.8. What is the percentage split of total revenue by category?

    SELECT category_id,
    	   category_name,
    	   ROUND(100*SUM(qty * s.price)::numeric / (SELECT SUM(qty * price) FROM sales_monthly), 2) AS revenue_percentage
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2;    


--3.9. What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)

    SELECT prod_id,
    	   product_name,
    	   ROUND(COUNT(txn_id)::numeric / (SELECT COUNT(DISTINCT txn_id) FROM sales_monthly), 3) AS txn_penetration
    FROM sales_monthly s
    LEFT JOIN balanced_tree.product_details p ON s.prod_id = p.product_id
    GROUP BY 1, 2;


--3.10. What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

    SELECT s.prod_id, t1.prod_id, t2.prod_id, COUNT(*) AS combination_cnt	   
    FROM sales_monthly s
    JOIN sales_monthly t1 ON t1.txn_id = s.txn_id 
    AND s.prod_id < t1.prod_id
    JOIN sales_monthly t2 ON t2.txn_id = s.txn_id
    AND t1.prod_id < t2.prod_id
    GROUP BY 1, 2, 3
    ORDER BY 4 DESC;    

---